In [ ]:
class mlp:
    '''Wielowarstwowy perceptron (1 warstwa ukryta)'''
    def __init__(self, n_in, n_out, n_hid):
        
        # liczba neuronów w kolejnych warstwach
        self.n_in=n_in
        self.n_hid=n_hid
        self.n_out=n_out

        # stany neuronów
        self.S_in=zeros(n_in+1)
        self.S_hid=zeros(n_hid)
        self.S_out=zeros(n_out)
        self.S_in[-1]=1

        # pola lokalne
        self.h_hid=zeros(n_hid)
        self.h_out=zeros(n_out)

        # wagi połączeń
        self.W_ih=zeros((n_in+1,n_hid))
        self.W_ho=zeros((n_hid,n_out))

        # delty
        self.dW_ih=zeros((n_in+1,n_hid))
        self.dW_ho=zeros((n_hid,n_out))
        
        self.reset()
    
    def reset(self):
        self.W_ih = normal(0, 1/sqrt(self.n_hid), self.W_ih.shape)
        self.W_ho = normal(0, 1/sqrt(self.n_out), self.W_ho.shape)
        
        
    def f(self, x):
        '''Funkcja aktywacji'''
#         return 1/(1+exp(-x))
#         return (tanh(x/2)+1)/2
        return maximum(x,0) # ReLU - rectified linear unit

    def Df(self, x):
        '''Pochodna funkcji aktywacji'''
#         y=self.f(x)
#         return y*(1-y)
#         return cosh(x)**-1/4
        return heaviside(x,0)

    def feed(self, inp):
        '''Przekazuje wektor danych do warstwy wejściowej'''
        self.S_in[:self.n_in]=inp

    def forward(self):
        '''Propaguje sygnał poprzez kolejne warstwy sieci'''
        self.h_hid = self.S_in @ self.W_ih
        self.S_hid = self.f(self.h_hid)
        self.h_out = self.S_hid @ self.W_ho
        self.S_out = self.f(self.h_out)
        
    def diff(self, p):
        return self.trainData[p,self.n_in:] - self.S_out

    def error(self,p):
        return sum(self.diff(p)**2)/2
    
    def setTrainData(self,data):
        '''Zapamiętuje referencję do wektorów uczących'''
        self.trainData=data

    def eval(self,inp):
        self.feed(inp)
        self.forward()
        return self.S_out


    def train(self, eta):
        '''Jedna epoka.'''
    
        Er=0
        p_num=self.trainData.shape[0]

        self.dW_ih.fill(0)
        self.dW_ho.fill(0)

        for p in range(p_num):
            self.feed(self.trainData[p,:self.n_in])
            self.forward()
        
            self.delta_out = self.Df(self.h_out)*self.diff(p)
            self.dW_ho += outer(self.S_hid, self.delta_out)
            self.delta_hid = self.Df(self.h_hid)*(self.W_ho @ self.delta_out)
            self.dW_ih += outer(self.S_in, self.delta_hid)

            Er+=self.error(p)

        self.W_ih+=eta*self.dW_ih
        self.W_ho+=eta*self.dW_ho
        return Er   